In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pickle
from pprint import pprint
from tqdm import tqdm
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.model_selection import KFold, GridSearchCV
import optuna

from utils import load_data_from_directory, split_into_K_folds, get_metrics
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")


/home/georg/projects/university/C2_Optimization_and_Inference/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Use a custom function to read images from the directory and assign labels based on the name of the subfolder they are in
training_files, training_labels = load_data_from_directory('../MIT_split/train/')
test_files, test_labels = load_data_from_directory('../MIT_split/test/')

In [3]:
# Since we want to do k-fold cross validation, we need to merge the training and test data and then re-split it into k folds

all_data = training_files + test_files
all_labels = training_labels + test_labels

k = 5  # number of folds
# split data into 5 folds using custom function
folds = split_into_K_folds(all_data, all_labels, k)

# Now each fold in the 'folds' list contains the training and test data for that fold in the format: (train_data, train_labels, test_data, test_labels)
for fold_index, fold in enumerate(folds):
    train_data, train_labels, test_data, test_labels = fold
    print(f"Fold {fold_index + 1}:")
    print(f"Number of samples in train set: {len(train_data)}")
    print(f"Number of samples in test set: {len(test_data)}")
    print(f"Number of labels in train set: {len(train_labels)}")
    print(f"Number of labels in test set: {len(test_labels)}")
    print()

Fold 1:
Number of samples in train set: 2150
Number of samples in test set: 538
Number of labels in train set: 2150
Number of labels in test set: 538

Fold 2:
Number of samples in train set: 2150
Number of samples in test set: 538
Number of labels in train set: 2150
Number of labels in test set: 538

Fold 3:
Number of samples in train set: 2150
Number of samples in test set: 538
Number of labels in train set: 2150
Number of labels in test set: 538

Fold 4:
Number of samples in train set: 2151
Number of samples in test set: 537
Number of labels in train set: 2151
Number of labels in test set: 537

Fold 5:
Number of samples in train set: 2151
Number of samples in test set: 537
Number of labels in train set: 2151
Number of labels in test set: 537



In [4]:
# Define Dense SIFT and Dense KAZE descriptors
def dense_sift(image, step_size=8, kp_scale=16):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    dense_sift = cv2.SIFT_create()
    keypoints = [cv2.KeyPoint(x, y, kp_scale) for y in range(0, image.shape[0], step_size) for x in range(0, image.shape[1], step_size)]
    _, descriptor = dense_sift.compute(image, keypoints)
    return descriptor

# generate descriptors for all images and store in dictionary to access them later in each fold without having to recompute them
def generate_descriptors(image_paths, descriptor_type, **kwargs):
    descriptors = {}
    for image_path in tqdm(image_paths):
        image = cv2.imread(image_path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        if descriptor_type == 'sift':
            sift = cv2.SIFT_create()
            _, descriptor = sift.detectAndCompute(gray_image, None)
        elif descriptor_type == 'kaze':
            kaze = cv2.KAZE_create(threshold=kwargs['threshold'])
            _, descriptor = kaze.detectAndCompute(gray_image, None)
        elif descriptor_type == 'dense_sift':
            descriptor = dense_sift(image, step_size=kwargs['step_size'], kp_scale=kwargs['kp_scale'])
        else:
            raise Exception('Unknown descriptor type')
        descriptors[image_path] = descriptor
    return descriptors
# we generate the descriptors for all images in the dataset and store them in a dictionary
# we use dense-SIFT with a step size of 8 and a keypoint scale of 12 as these values gave the best results in our experiments
descriptors_dictionary = generate_descriptors(all_data, 'dense_sift', step_size=8, kp_scale=12)


100%|██████████| 2688/2688 [02:31<00:00, 17.78it/s]


In [5]:
# Define a function to create a dictionary linking a histogram of visual words to the corresponding images images:
def create_visual_words(image_paths, descriptors_dictionary, codebook):
    visual_words = {}
    for image_path in image_paths:
        descriptor = descriptors_dictionary[image_path]
        labels = codebook.predict(descriptor)
        visual_words[image_path] = np.bincount(labels, minlength=codebook.n_clusters)
    return np.array(list(visual_words.values()))


In [6]:
# initialize a dictionary to store the codebooks for each fold
codebooks = {'fold_0': {}, 'fold_1': {}, 'fold_2': {}, 'fold_3': {}, 'fold_4': {}}

# define the codebook sizes we want to test
codebook_sizes = [32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384]

# loop over folds
for i, fold in enumerate(folds):
    train_data, train_labels, test_data, test_labels = fold
    train_descriptors = []

    # fetch the descriptors for the training images in the current fold
    for file in train_data:
        train_descriptors.append(descriptors_dictionary[file])

    # stack the descriptors vertically to create a single numpy array for clustering
    train_D = np.vstack(list(train_descriptors)) 

    # loop over codebook sizes
    for k in tqdm(codebook_sizes, desc=f'Creating Codebooks for fold {i}'):
        # initialize the codebook
        codebook = MiniBatchKMeans(n_clusters=k, 
                           verbose=False, 
                           batch_size=2150,
                           compute_labels=False,
                           reassignment_ratio=0.01,
                           random_state=42)
        # fit the codebook
        codebook.fit(train_D)
        # store the codebook in the dictionary
        codebooks[f'fold_{i}'][k] = codebook


Creating Codebooks for fold 4: 100%|██████████| 10/10 [1:19:34<00:00, 477.50s/it] 


### Gridsearch for KNN parameters ###

In [12]:
def objective(trial):

    n_clusters = trial.suggest_categorical('n_clusters', codebook_sizes)
    n_neighbors = trial.suggest_int('n_neighbors', 1, 25)
    metric = trial.suggest_categorical('metric', ['euclidean', 
                                                    'minkowski', 
                                                    'cosine', 
                                                    'manhattan', 
                                                    'hamming'])
    weight = trial.suggest_categorical('weight', ['uniform', 'distance'])

    metrics = []
    # Run k-fold cross validation per hyperparameter combination
    for i, fold in enumerate(folds, 0):

        train_data, train_labels, test_data, test_labels = fold
        
        codebook = codebooks[f'fold_{i}'][n_clusters]
        # create visual words for training data
        train_visual_words = create_visual_words(train_data, descriptors_dictionary, codebook)
        # create visual words for test data
        test_visual_words = create_visual_words(test_data, descriptors_dictionary, codebook)
        
        # fit kNN classifier
        print(f'Fold {i}: fitting knn with n_neighbors={n_neighbors}, metric={metric}, weights={weight}')
        knn = KNeighborsClassifier(n_neighbors=n_neighbors,n_jobs=-1,metric=metric, weights=weight)
        knn.fit(train_visual_words, train_labels)
        # predict labels for test data
        pred_labels = knn.predict(test_visual_words)

        # get accuracy metric for current run
        metrics.append(get_metrics(test_labels, pred_labels)[0])

    return sum(metrics)/len(metrics)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

study.best_params

[I 2024-01-05 12:31:08,102] A new study created in memory with name: no-name-23b00054-5a9d-4b97-ba40-888bc8564a10


Fold 0: fitting knn with n_neighbors=12, metric=manhattan, weights=uniform
Fold 1: fitting knn with n_neighbors=12, metric=manhattan, weights=uniform
Fold 2: fitting knn with n_neighbors=12, metric=manhattan, weights=uniform
Fold 3: fitting knn with n_neighbors=12, metric=manhattan, weights=uniform


[I 2024-01-05 12:31:19,950] Trial 0 finished with value: 0.8117498425093282 and parameters: {'classifier': 'knn', 'n_clusters': 256, 'n_neighbors': 12, 'metric': 'manhattan', 'weight': 'uniform'}. Best is trial 0 with value: 0.8117498425093282.


Fold 4: fitting knn with n_neighbors=12, metric=manhattan, weights=uniform
Fold 0: fitting knn with n_neighbors=21, metric=manhattan, weights=uniform
Fold 1: fitting knn with n_neighbors=21, metric=manhattan, weights=uniform
Fold 2: fitting knn with n_neighbors=21, metric=manhattan, weights=uniform
Fold 3: fitting knn with n_neighbors=21, metric=manhattan, weights=uniform
Fold 4: fitting knn with n_neighbors=21, metric=manhattan, weights=uniform


[I 2024-01-05 12:39:58,096] Trial 1 finished with value: 0.8039466123929582 and parameters: {'classifier': 'knn', 'n_clusters': 16384, 'n_neighbors': 21, 'metric': 'manhattan', 'weight': 'uniform'}. Best is trial 0 with value: 0.8117498425093282.


...

[I 2024-01-05 14:23:34,275] Trial 98 finished with value: 0.808771711214028 and parameters: {'classifier': 'knn', 'n_clusters': 1024, 'n_neighbors': 3, 'metric': 'manhattan', 'weight': 'uniform'}. Best is trial 41 with value: 0.8258873128283941.


Fold 0: fitting knn with n_neighbors=7, metric=hamming, weights=distance
Fold 1: fitting knn with n_neighbors=7, metric=hamming, weights=distance
Fold 2: fitting knn with n_neighbors=7, metric=hamming, weights=distance
Fold 3: fitting knn with n_neighbors=7, metric=hamming, weights=distance


[I 2024-01-05 14:23:53,384] Trial 99 finished with value: 0.30952351283808577 and parameters: {'classifier': 'knn', 'n_clusters': 512, 'n_neighbors': 7, 'metric': 'hamming', 'weight': 'distance'}. Best is trial 41 with value: 0.8258873128283941.


Fold 4: fitting knn with n_neighbors=7, metric=hamming, weights=distance


{'classifier': 'knn',
 'n_clusters': 512,
 'n_neighbors': 6,
 'metric': 'manhattan',
 'weight': 'distance'}

### Gridsearch for SVC parameters ###

In [13]:
def objective(trial):

    n_clusters = trial.suggest_categorical('n_clusters', codebook_sizes)

    svm_kernel = trial.suggest_categorical('svm_kernel', ['rbf', 'poly', 'sigmoid'])
    svm_C = trial.suggest_categorical('svm_C', [0.01, 0.1, 1, 10, 100])
    svm_gamma = trial.suggest_categorical('svm_gamma', [0.05, 0.1, 0.15, 0.2, 0.25])

    metrics = []
    # Run k-fold cross validation per hyperparameter combination
    for i, fold in enumerate(folds, 0):

        train_data, train_labels, test_data, test_labels = fold
        
        codebook = codebooks[f'fold_{i}'][n_clusters]
        # create visual words for training data
        train_visual_words = create_visual_words(train_data, descriptors_dictionary, codebook)
        # create visual words for test data
        test_visual_words = create_visual_words(test_data, descriptors_dictionary, codebook)
        
        
        # fit SVM classifier
        print(f'Fold {i}: fitting svm with C={svm_C}, kernel={svm_kernel}, gamma={svm_gamma}, n_clusters={n_clusters}')
        svm = SVC(C=svm_C, kernel=svm_kernel, gamma=svm_gamma)
        svm.fit(train_visual_words, train_labels)
        # predict labels for test data
        pred_labels = svm.predict(test_visual_words)
    
        # get accuracy metric for current run
        metrics.append(get_metrics(test_labels, pred_labels)[0])

    return sum(metrics)/len(metrics)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

study.best_params

[I 2024-01-05 18:58:13,226] A new study created in memory with name: no-name-1496ce56-3621-4308-939a-1b7fd28d6d1b


Fold 0: fitting svm with C=0.1, kernel=sigmoid, gamma=0.05, n_clusters=1024
Fold 1: fitting svm with C=0.1, kernel=sigmoid, gamma=0.05, n_clusters=1024
Fold 2: fitting svm with C=0.1, kernel=sigmoid, gamma=0.05, n_clusters=1024
Fold 3: fitting svm with C=0.1, kernel=sigmoid, gamma=0.05, n_clusters=1024
Fold 4: fitting svm with C=0.1, kernel=sigmoid, gamma=0.05, n_clusters=1024


[I 2024-01-05 18:59:01,688] Trial 0 finished with value: 0.15847507493786908 and parameters: {'n_clusters': 1024, 'svm_kernel': 'sigmoid', 'svm_C': 0.1, 'svm_gamma': 0.05}. Best is trial 0 with value: 0.15847507493786908.


Fold 0: fitting svm with C=1, kernel=sigmoid, gamma=0.05, n_clusters=4096
Fold 1: fitting svm with C=1, kernel=sigmoid, gamma=0.05, n_clusters=4096
Fold 2: fitting svm with C=1, kernel=sigmoid, gamma=0.05, n_clusters=4096
Fold 3: fitting svm with C=1, kernel=sigmoid, gamma=0.05, n_clusters=4096
Fold 4: fitting svm with C=1, kernel=sigmoid, gamma=0.05, n_clusters=4096


[I 2024-01-05 19:02:16,225] Trial 1 finished with value: 0.3467356164288731 and parameters: {'n_clusters': 4096, 'svm_kernel': 'sigmoid', 'svm_C': 1, 'svm_gamma': 0.05}. Best is trial 1 with value: 0.3467356164288731.


...

Fold 4: fitting svm with C=100, kernel=poly, gamma=0.1, n_clusters=64
Fold 0: fitting svm with C=0.1, kernel=poly, gamma=0.1, n_clusters=4096
Fold 1: fitting svm with C=0.1, kernel=poly, gamma=0.1, n_clusters=4096
Fold 2: fitting svm with C=0.1, kernel=poly, gamma=0.1, n_clusters=4096
Fold 3: fitting svm with C=0.1, kernel=poly, gamma=0.1, n_clusters=4096
Fold 4: fitting svm with C=0.1, kernel=poly, gamma=0.1, n_clusters=4096


[I 2024-01-05 19:52:54,632] Trial 19 finished with value: 0.643603801928655 and parameters: {'n_clusters': 4096, 'svm_kernel': 'poly', 'svm_C': 0.1, 'svm_gamma': 0.1}. Best is trial 8 with value: 0.8385371020331872.


Fold 0: fitting svm with C=0.01, kernel=rbf, gamma=0.25, n_clusters=8192


[W 2024-01-05 19:54:55,538] Trial 20 failed with parameters: {'n_clusters': 8192, 'svm_kernel': 'rbf', 'svm_C': 0.01, 'svm_gamma': 0.25} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/georg/projects/university/C2_Optimization_and_Inference/venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_49809/553763206.py", line 17, in objective
    train_visual_words = create_visual_words(train_data, descriptors_dictionary, codebook)
  File "/tmp/ipykernel_49809/4024483382.py", line 6, in create_visual_words
    labels = codebook.predict(descriptor)
  File "/home/georg/projects/university/C2_Optimization_and_Inference/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py", line 1111, in predict
    labels = _labels_inertia_threadpool_limit(
  File "/home/georg/projects/university/C2_Optimization_and_Inference/venv/lib/python3.10/site-packages/skle

KeyboardInterrupt: 

In [26]:
# initialize a dictionary to store the codebooks for each fold
codebooks = {'fold_0': {}, 'fold_1': {}, 'fold_2': {}, 'fold_3': {}, 'fold_4': {}}

# define the codebook sizes we want to test
codebook_sizes = [32, 64, 128, 256, 512, 1024]

# loop over folds
for i, fold in enumerate(folds):
    train_data, train_labels, test_data, test_labels = fold
    train_descriptors = []

    # fetch the descriptors for the training images in the current fold
    for file in train_data:
        train_descriptors.append(descriptors_dictionary[file])

    # stack the descriptors vertically to create a single numpy array for clustering
    train_D = np.vstack(list(train_descriptors)) 

    # loop over codebook sizes
    for k in tqdm(codebook_sizes, desc=f'Creating Codebooks for fold {i}'):
        # initialize the codebook
        codebook = MiniBatchKMeans(n_clusters=k, 
                           verbose=False, 
                           batch_size=2150,
                           compute_labels=False,
                           reassignment_ratio=0.01,
                           random_state=42)
        # fit the codebook
        codebook.fit(train_D)
        # store the codebook in the dictionary
        codebooks[f'fold_{i}'][k] = codebook

Creating Codebooks for fold 4: 100%|██████████| 6/6 [01:08<00:00, 11.38s/it]


The previous kernel has been interrupted because it was run on a different device and took too long to re-run. The outputs can be found in SVC_gridsearch.ipynb

### Gridsearch PCA parameters for KNN ###

In [27]:
pca_param = [16, 32, 64, 128, 256]
n_clusters = 512
n_neighbors = 6
distance = 'manhattan'
weights = 'distance'

for val in pca_param:
    pca_accuracies = []
    print(f"\nPCA Components = {val}")
    for i, fold in enumerate(folds, 0):

        train_data, train_labels, test_data, test_labels = fold
        
        codebook = codebooks[f'fold_{i}'][n_clusters]
        # create visual words for training data
        train_visual_words = create_visual_words(train_data, descriptors_dictionary, codebook)
        # create visual words for test data
        test_visual_words = create_visual_words(test_data, descriptors_dictionary, codebook)

        pca = PCA(n_components=int(val))
        VWpca = pca.fit_transform(train_visual_words)
        knnpca = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, metric=distance)
        knnpca.fit(VWpca, train_labels) 
        vwtestpca = pca.transform(test_visual_words)
        accuracy = 100*knnpca.score(vwtestpca, test_labels)

        print(f'Fold {i} accuracy: {accuracy}')
        pca_accuracies.append(accuracy)

    print('Average Accuracy: ', np.mean(pca_accuracies))


PCA Components = 16
Fold 0 accuracy: 76.95167286245353
Fold 1 accuracy: 79.5539033457249
Fold 2 accuracy: 73.97769516728626
Fold 3 accuracy: 77.09497206703911
Fold 4 accuracy: 75.23277467411546
Average Accuracy:  76.56220362332385

PCA Components = 32
Fold 0 accuracy: 79.182156133829
Fold 1 accuracy: 79.73977695167285
Fold 2 accuracy: 77.32342007434944
Fold 3 accuracy: 76.16387337057728
Fold 4 accuracy: 78.02607076350093
Average Accuracy:  78.08705945878589

PCA Components = 64
Fold 0 accuracy: 79.36802973977694
Fold 1 accuracy: 77.88104089219331
Fold 2 accuracy: 74.90706319702602
Fold 3 accuracy: 74.86033519553072
Fold 4 accuracy: 76.35009310986965
Average Accuracy:  76.67331242687933

PCA Components = 128
Fold 0 accuracy: 75.46468401486989
Fold 1 accuracy: 77.32342007434944
Fold 2 accuracy: 73.23420074349443
Fold 3 accuracy: 75.41899441340783
Fold 4 accuracy: 76.35009310986965
Average Accuracy:  75.55827847119825

PCA Components = 256
Fold 0 accuracy: 75.09293680297398
Fold 1 accura

### Gridsearch LDA parameters for KNN ###

In [28]:

lda_param = [1, 3, 5, 7]
      
for val in lda_param:
        lda_accuracies = []
        print(f"\nLDA Components = {val}")
        for i, fold in enumerate(folds, 0):

                train_data, train_labels, test_data, test_labels = fold
                
                codebook = codebooks[f'fold_{i}'][n_clusters]
                # create visual words for training data
                train_visual_words = create_visual_words(train_data, descriptors_dictionary, codebook)
                # create visual words for test data
                test_visual_words = create_visual_words(test_data, descriptors_dictionary, codebook)

                
                lda = LinearDiscriminantAnalysis(n_components=val)
                VWlda = lda.fit_transform(train_visual_words, train_labels)
                knnlda = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, metric=distance)
                knnlda.fit(VWlda, train_labels) 
                vwtestlda = lda.transform(test_visual_words)
                accuracy = 100*knnlda.score(vwtestlda, test_labels)

                
                print(f'Fold {i} accuracy: {accuracy}')
                lda_accuracies.append(accuracy)

        print('Average Accuracy: ', np.mean(lda_accuracies))


LDA Components = 1
Fold 0 accuracy: 35.87360594795539
Fold 1 accuracy: 29.739776951672862
Fold 2 accuracy: 36.43122676579926
Fold 3 accuracy: 32.774674115456236
Fold 4 accuracy: 30.353817504655495
Average Accuracy:  33.034620257107846

LDA Components = 3
Fold 0 accuracy: 72.11895910780669
Fold 1 accuracy: 70.4460966542751
Fold 2 accuracy: 70.817843866171
Fold 3 accuracy: 70.94972067039106
Fold 4 accuracy: 68.15642458100558
Average Accuracy:  70.49780897592989

LDA Components = 5
Fold 0 accuracy: 81.78438661710037
Fold 1 accuracy: 81.22676579925651
Fold 2 accuracy: 80.29739776951673
Fold 3 accuracy: 78.2122905027933
Fold 4 accuracy: 80.07448789571696
Average Accuracy:  80.31906571687678

LDA Components = 7
Fold 0 accuracy: 85.50185873605948
Fold 1 accuracy: 83.45724907063197
Fold 2 accuracy: 81.59851301115242
Fold 3 accuracy: 81.75046554934823
Fold 4 accuracy: 82.68156424581005
Average Accuracy:  82.99793012260042


### Gridsearch PCA parameters for SVM ###

In [29]:
#best combination parameters
n_clusters = 1024
pca_param = [16, 32, 64, 128, 256]
svc_C = 0.01
svc_gamma = 0.2
svc_kernel = 'linear'           

for val in pca_param:
        pca_accuracies = []
        print(f"\nPCA Components = {val}")
        for i, fold in enumerate(folds, 0):

                train_data, train_labels, test_data, test_labels = fold
                
                codebook = codebooks[f'fold_{i}'][n_clusters]
                # create visual words for training data
                train_visual_words = create_visual_words(train_data, descriptors_dictionary, codebook)
                # create visual words for test data
                test_visual_words = create_visual_words(test_data, descriptors_dictionary, codebook)


                pca = PCA(n_components=val)
                VWpca = pca.fit_transform(train_visual_words)
                svcpca = SVC(kernel=svc_kernel, C=svc_C, gamma=svc_gamma)
                svcpca.fit(VWpca, train_labels) 
                vwtestpca = pca.transform(test_visual_words)
                accuracy = 100*svcpca.score(vwtestpca, test_labels)

                print(f'Fold {i} accuracy: {accuracy}')
                pca_accuracies.append(accuracy)

        print('Average Accuracy: ', np.mean(pca_accuracies))


PCA Components = 16
Fold 0 accuracy: 83.08550185873605
Fold 1 accuracy: 85.31598513011153
Fold 2 accuracy: 78.81040892193309
Fold 3 accuracy: 79.14338919925513
Fold 4 accuracy: 79.70204841713222
Average Accuracy:  81.2114667054336

PCA Components = 32
Fold 0 accuracy: 83.27137546468401
Fold 1 accuracy: 85.87360594795538
Fold 2 accuracy: 80.29739776951673
Fold 3 accuracy: 81.00558659217877
Fold 4 accuracy: 81.00558659217877
Average Accuracy:  82.29071047330272

PCA Components = 64
Fold 0 accuracy: 85.31598513011153
Fold 1 accuracy: 83.27137546468401
Fold 2 accuracy: 80.66914498141264
Fold 3 accuracy: 83.9851024208566
Fold 4 accuracy: 82.12290502793296
Average Accuracy:  83.07290260499954

PCA Components = 128
Fold 0 accuracy: 86.05947955390334
Fold 1 accuracy: 84.20074349442379
Fold 2 accuracy: 81.59851301115242
Fold 3 accuracy: 82.49534450651768
Fold 4 accuracy: 83.42644320297951
Average Accuracy:  83.55610475379535

PCA Components = 256
Fold 0 accuracy: 86.80297397769516
Fold 1 accur

### Gridsearch LDA parameters for SVM ###

In [30]:
#best combination parameters
n_clusters = 1024
lda_param = [1, 3, 5, 7]
svc_C = 0.01
svc_gamma = 0.2
svc_kernel = 'linear'           

for val in lda_param:
        lda_accuracies = []
        print(f"\nLDA Components = {val}")
        for i, fold in enumerate(folds, 0):

                train_data, train_labels, test_data, test_labels = fold
                
                codebook = codebooks[f'fold_{i}'][n_clusters]
                # create visual words for training data
                train_visual_words = create_visual_words(train_data, descriptors_dictionary, codebook)
                # create visual words for test data
                test_visual_words = create_visual_words(test_data, descriptors_dictionary, codebook)

                
                lda = LinearDiscriminantAnalysis(n_components=val)
                VWlda = lda.fit_transform(train_visual_words, train_labels)
                svclda = SVC(kernel=svc_kernel, C=svc_C, gamma=svc_gamma)
                svclda.fit(VWlda, train_labels) 
                vwtestlda = lda.transform(test_visual_words)
                accuracy = 100*svclda.score(vwtestlda, test_labels)

                
                print(f'Fold {i} accuracy: {accuracy}')
                lda_accuracies.append(accuracy)

        print('Average Accuracy: ', np.mean(lda_accuracies))


LDA Components = 1
Fold 0 accuracy: 36.2453531598513
Fold 1 accuracy: 29.182156133828997
Fold 2 accuracy: 36.43122676579926
Fold 3 accuracy: 33.33333333333333
Fold 4 accuracy: 32.02979515828678
Average Accuracy:  33.44437291021993

LDA Components = 3
Fold 0 accuracy: 70.4460966542751
Fold 1 accuracy: 67.84386617100373
Fold 2 accuracy: 65.05576208178438
Fold 3 accuracy: 63.128491620111724
Fold 4 accuracy: 67.59776536312849
Average Accuracy:  66.81439637806068

LDA Components = 5
Fold 0 accuracy: 79.73977695167285
Fold 1 accuracy: 77.5092936802974
Fold 2 accuracy: 75.8364312267658
Fold 3 accuracy: 73.9292364990689
Fold 4 accuracy: 74.11545623836126
Average Accuracy:  76.22603891923325

LDA Components = 7
Fold 0 accuracy: 82.15613382899627
Fold 1 accuracy: 76.95167286245353
Fold 2 accuracy: 79.5539033457249
Fold 3 accuracy: 76.72253258845437
Fold 4 accuracy: 81.00558659217877
Average Accuracy:  79.27796584356157


### Test Spatial Pyramids for KNN ###

In [31]:
codebooks = {'fold_0': {}, 'fold_1': {}, 'fold_2': {}, 'fold_3': {}, 'fold_4': {}}
codebook_size = 512
levels = 3

Training_descriptors = {}

print(f'Spatial Pyramid with {levels} levels.')

for filename in tqdm(all_data):

    image = cv2.imread(filename)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    descriptors = []

    for level in range(levels):
        # Calculate the size of each cell in the pyramid
        cell_size = (image.shape[0] // (2 ** level), image.shape[1] // (2 ** level))

        # Divide the image into cells
        for i in range(0, image.shape[0], cell_size[0]):
            for j in range(0, image.shape[1], cell_size[1]):
                cell = image[i:i+cell_size[0], j:j+cell_size[1]]  # Extract cell
                # Perform feature extraction (You can use any desired method here, e.g., SIFT, SURF, HOG)
                # Example: using ORB feature detector
                descriptor = dense_sift(cell, step_size=8, kp_scale=12)
                if descriptor is not None:
                    descriptors.append(descriptor)  # Store descriptors for each cell

    # Combine features (Example: using concatenation)
    combined_features = np.concatenate(descriptors, axis=0)
    
    Training_descriptors[filename] = combined_features

metrics = []

for i, fold in enumerate(folds):
    train_data, train_labels, test_data, test_labels = fold
    train_descriptors = []

    for file in train_data:
        train_descriptors.append(Training_descriptors[file])
    train_D = np.vstack(list(train_descriptors)) 


    codebook = MiniBatchKMeans(n_clusters=codebook_size, 
                        verbose=False, 
                        batch_size=2150,
                        compute_labels=False,
                        reassignment_ratio=0.01,
                        random_state=42)
    
    codebook.fit(train_D)

    train_visual_words = create_visual_words(train_data, descriptors_dictionary, codebook)
    test_visual_words = create_visual_words(test_data, descriptors_dictionary, codebook)
    
    knn = KNeighborsClassifier(n_neighbors=6, n_jobs=-1, metric='manhattan', weights='distance')
    knn.fit(train_visual_words, train_labels)
    pred_labels = knn.predict(test_visual_words)

    accuracy = get_metrics(test_labels, pred_labels)[0]
    print(f'Fold {i} accuracy: {accuracy}')
    metrics.append(accuracy)

print('Average accuracy', np.mean(metrics))


Spatial Pyramid with 3 levels.


  0%|          | 0/2688 [00:00<?, ?it/s]

100%|██████████| 2688/2688 [04:40<00:00,  9.58it/s]


Fold 0 accuracy: 0.8327137546468402
Fold 1 accuracy: 0.8234200743494424
Fold 2 accuracy: 0.7973977695167286
Fold 3 accuracy: 0.7970204841713222
Fold 4 accuracy: 0.8286778398510242
Average accuracy 0.8158459845070715


### Test Spatial Pyramids for SVC ###

In [32]:
codebooks = {'fold_0': {}, 'fold_1': {}, 'fold_2': {}, 'fold_3': {}, 'fold_4': {}}
codebook_size = 1024
levels = 3

Training_descriptors = {}

print(f'Spatial Pyramid with {levels} levels.')

for filename in tqdm(all_data):

    image = cv2.imread(filename)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    descriptors = []

    for level in range(levels):
        # Calculate the size of each cell in the pyramid
        cell_size = (image.shape[0] // (2 ** level), image.shape[1] // (2 ** level))

        # Divide the image into cells
        for i in range(0, image.shape[0], cell_size[0]):
            for j in range(0, image.shape[1], cell_size[1]):
                cell = image[i:i+cell_size[0], j:j+cell_size[1]]  # Extract cell
                # Perform feature extraction (You can use any desired method here, e.g., SIFT, SURF, HOG)
                # Example: using ORB feature detector
                descriptor = dense_sift(cell, step_size=8, kp_scale=12)
                if descriptor is not None:
                    descriptors.append(descriptor)  # Store descriptors for each cell

    # Combine features (Example: using concatenation)
    combined_features = np.concatenate(descriptors, axis=0)
    
    Training_descriptors[filename] = combined_features
    
metrics = []
for i, fold in enumerate(folds):
    train_data, train_labels, test_data, test_labels = fold
    train_descriptors = []

    for file in train_data:
        train_descriptors.append(Training_descriptors[file])
    train_D = np.vstack(list(train_descriptors)) 


    codebook = MiniBatchKMeans(n_clusters=codebook_size, 
                        verbose=False, 
                        batch_size=2150,
                        compute_labels=False,
                        reassignment_ratio=0.01,
                        random_state=42)
    
    codebook.fit(train_D)

    train_visual_words = create_visual_words(train_data, descriptors_dictionary, codebook)
    test_visual_words = create_visual_words(test_data, descriptors_dictionary, codebook)
    
    svm = SVC(kernel=svc_kernel, C=svc_C, gamma=svc_gamma)
    svm.fit(train_visual_words, train_labels)

    pred_labels = svm.predict(test_visual_words)

    accuracy = get_metrics(test_labels, pred_labels)[0]
    print(f'Fold {i} accuracy: {accuracy}')
    metrics.append(accuracy)

print('Average accuracy', np.mean(metrics))

Spatial Pyramid with 3 levels.


100%|██████████| 2688/2688 [04:41<00:00,  9.53it/s]


Fold 0 accuracy: 0.8568773234200744
Fold 1 accuracy: 0.8513011152416357
Fold 2 accuracy: 0.828996282527881
Fold 3 accuracy: 0.8286778398510242
Fold 4 accuracy: 0.8342644320297952
Average accuracy 0.840023398614082
